In [1]:
import shapefile
import argparse
import numpy as np
import matplotlib.pyplot as plt
from copy import copy
import os

In [2]:
lat_deg = 110950
long_deg = 85000

In [3]:
path = "/home/jovyan/shared/max/rf_rt/helper_files/boulder_campus/holey_Bs/boulder_bs/"

In [4]:
select_buildings = [1346048]

In [5]:
shapefile_f = "Buildings/Building_Footprints.shp"

In [6]:
sf = shapefile.Reader(shapefile_f)

In [7]:
shapes = sf.shapes()
rec = sf.records()

In [8]:
tmp_vertex_degrees = []
a_building_height = []
a_building_low = []
a_building_id = []
a_holey_building = []

In [9]:
for i in range(len(shapes)):
    if rec[i]['DRCOGID'] in select_buildings:
        print("Building is {}".format(rec[i]['DRCOGID']))
        print(shapes[i].points)
        print("-----------------------")
        print(shapes[i].parts)
        tmp = []
        for vert in shapes[i].points:
            tmp.append(np.array([copy(vert[0]),copy(vert[1])]))
        if len(shapes[i].parts) > 1: ## marks building with holes
            a_holey_building.append(shapes[i].parts)
        else:
            a_holey_building.append(False)
        tmp_vertex_degrees.append(tmp)
        a_building_height.append(rec[i]['BLDGHEIGHT']+rec[i]['GROUNDELEV'])
        a_building_low.append(rec[i]['GROUNDELEV'])
        a_building_id.append(rec[i]['DRCOGID'])
        # if (len(a_building_id)> 10500):
        #     break

Building is 1346048
[(-105.27437682057574, 40.007883281009676), (-105.27437653711377, 40.0078946160663), (-105.2743137450515, 40.00789368890754), (-105.2743160514556, 40.00780124595448), (-105.27437884460993, 40.007802171312946), (-105.27437778864194, 40.00784450531428), (-105.27437682057574, 40.007883281009676)]
-----------------------
[0]


In [10]:
a_vertex_degrees = np.array(tmp_vertex_degrees, dtype=object)

In [11]:
print("num buildings to write: {}".format(len(a_vertex_degrees)))
print("total number of objects: {}".format(a_vertex_degrees.size))

num buildings to write: 1
total number of objects: 14


In [20]:
## longitude, latitude
lower_left = np.array([-105.27429, 40.00643])
upper_right = np.array([-105.27377, 40.00832])
upper_left = np.array([-105.27512, 40.00839])

In [21]:
vc = 0
bc = 0

xx = []
yy = []

In [22]:
for i in range(len(a_vertex_degrees)):
    xx = []
    yy = []
    print("building height is ", a_building_height[i])
    building_high = a_building_height[i] * 0.3048 ## convert from feet to meters
    building_low = a_building_low[i] * 0.3048 ## convert from feet to meters
    v_list = []
    f_list = []
    building_vc = 0
    meter_v = []
    for j in range(0, len(a_vertex_degrees[i])):
        xx.append((a_vertex_degrees[i][j][0] - upper_left[0]) * long_deg)
        yy.append((upper_left[1] - a_vertex_degrees[i][j][1]) * lat_deg)
        if j == 0:
            meter_v.append((a_vertex_degrees[i][j] - lower_left) * np.array([long_deg, lat_deg]))
            vertex = (a_vertex_degrees[i][j] - lower_left) * np.array([long_deg, lat_deg])
            v_list.append("v {:.4f} {:.4f} {:.4f}".format(vertex[1], building_low, vertex[0]))
            v_list.append("v {:.4f} {:.4f} {:.4f}".format(vertex[1], building_high, vertex[0]))
            vc += 2
            building_vc += 2
        elif j == len(a_vertex_degrees[i])-1:
            f_list.append("f {} {} {}".format(vc-1, vc-building_vc+1, vc-building_vc+2))
            f_list.append("f {} {} {}".format(vc-1, vc-building_vc+2, vc))
            # if a_holey_building[i]:
            #     meter_v.append((a_vertex_degrees[i][j] - lower_left) * np.array([long_deg, lat_deg]))
        else:
            meter_v.append((a_vertex_degrees[i][j] - lower_left) * np.array([long_deg, lat_deg]))
            vertex = (a_vertex_degrees[i][j] - lower_left) * np.array([long_deg, lat_deg])
            v_list.append("v {:.4f} {:.4f} {:.4f}".format(vertex[1], building_low, vertex[0]))
            v_list.append("v {:.4f} {:.4f} {:.4f}".format(vertex[1], building_high, vertex[0]))
            vc += 2
            building_vc += 2
            f_list.append("f {} {} {}".format(vc-3, vc-1, vc))
            f_list.append("f {} {} {}".format(vc-3, vc, vc-2))

building height is  5441


In [23]:
v_list

['v 161.2415 1655.9784 -7.3797',
 'v 161.2415 1658.4168 -7.3797',
 'v 162.4992 1655.9784 -7.3557',
 'v 162.4992 1658.4168 -7.3557',
 'v 162.3963 1655.9784 -2.0183',
 'v 162.3963 1658.4168 -2.0183',
 'v 152.1397 1655.9784 -2.2144',
 'v 152.1397 1658.4168 -2.2144',
 'v 152.2424 1655.9784 -7.5518',
 'v 152.2424 1658.4168 -7.5518',
 'v 156.9394 1655.9784 -7.4620',
 'v 156.9394 1658.4168 -7.4620']

In [24]:
f_list

['f 1 3 4',
 'f 1 4 2',
 'f 3 5 6',
 'f 3 6 4',
 'f 5 7 8',
 'f 5 8 6',
 'f 7 9 10',
 'f 7 10 8',
 'f 9 11 12',
 'f 9 12 10',
 'f 11 1 2',
 'f 11 2 12']

In [25]:
meter_v

[array([-7.379748938376451, 161.24152802331935], dtype=object),
 array([-7.355654670746503, 162.49915255604606], dtype=object),
 array([-2.0183293781173006, 162.39628429098474], dtype=object),
 array([-2.214373726516783, 152.13973864961935], dtype=object),
 array([-7.551791844377931, 152.2424071711594], dtype=object),
 array([-7.462034565648423, 156.93936461934152], dtype=object)]